In [1]:
import pandas as pd
import numpy as np
import os
import re
from csv import DictReader
import collections
from tqdm import tqdm
import json

import Bio
from Bio import SeqUtils

import pymol
from pymol import CmdException

In [2]:
WORKDIR = "/Users/jessicakaraguesian/Google Drive/My Drive/Dror Lab/projects/flexibility"
min_maxed_dir = WORKDIR+"/dataset/qfit_op"
raw_dir = WORKDIR+"/dataset/nonnormalized_op"
resolution_file = WORKDIR+"/dataset/res_table.csv"
fetch_path = WORKDIR+"/pdb_structures"
bfactor_file = WORKDIR+"/b_factors.json"
structure_dir = WORKDIR+"/dataset/renumber"

## Retrieve data

In [3]:
def extract_op_data(directory):
    list_of_dicts = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f) and filename.endswith('.out'):
            pdb_id = re.findall("([a-z0-9]{4})_",filename)
            if len(pdb_id) != 1:
                print(filename)
                break
            pdb_id = pdb_id[0]
            new_dicts = list(DictReader(open(f,"r")))
            new_dicts = [dict(d,**{'pdb_id':pdb_id}) for d in new_dicts]
            list_of_dicts += new_dicts
            
    df = pd.DataFrame(list_of_dicts)
    cols = df.columns.tolist()[::-1]
    df = df[cols]
    
    return df

In [4]:
# Min-maxed data
df_min_maxed = extract_op_data(min_maxed_dir)
df_min_maxed = df_min_maxed.rename(columns={'resi':'qFit_resi', 'resn':'qFit_resn', 'chain':'qFit_chain'})
df_min_maxed['label'] = df_min_maxed.pdb_id+'_'+df_min_maxed.qFit_chain+"_"+df_min_maxed.qFit_resi
df_min_maxed.set_index('label',inplace=True)
df_min_maxed = df_min_maxed.astype({'s2ang':float,'s2ortho':float,'s2calc':float})

In [5]:
# Raw data
df_raw = extract_op_data(raw_dir)
df_raw = df_raw.rename(columns={'resi':'qFit_resi', 'resn':'qFit_resn', 'chain':'qFit_chain'})
df_raw.qFit_resn = df_raw.qFit_resn.apply(lambda x: Bio.SeqUtils.IUPACData.protein_letters_3to1[x.title()])
df_raw['label'] = df_raw.pdb_id+'_'+df_raw.qFit_chain+"_"+df_raw.qFit_resi
df_raw.set_index('label',inplace=True)
df_raw = df_raw.astype({'s2ang':float,'s2ortho':float,'s2calc':float})

In [6]:
# Resolution data
df_resolution = pd.read_csv(resolution_file)
df_resolution = df_resolution.drop_duplicates(subset=['pdb_id'])

In [7]:
# Get IDs of raw data with resolution available
min_maxed_list = list(set(df_min_maxed.pdb_id))
raw_list = list(set(df_raw.pdb_id))
res_list = list(set(df_resolution.pdb_id))
shared_ids = [a for a in raw_list if a in res_list]
print(len(min_maxed_list), len(raw_list), len(res_list), len(shared_ids))

106 1520 1113 1035


In [8]:
# We will not deal with the min-maxed data for now, since it is applied per-protein
# and doesn't make sense for learning over residues of many structures

In [9]:
# Limit raw data to those with resolution data available
df_raw = df_raw[df_raw.pdb_id.isin(shared_ids)]
# Set resolution for raw data
df_resolution.set_index('pdb_id',inplace=True)
df_raw['resolution'] = df_raw.pdb_id.apply(lambda x: df_resolution.resolution[x])

## Align and map to PDB structures 

In [10]:
def int_str_to_int(int_str):
    all_ints = re.findall(r'\d+', int_str)
    print("INT STRING:",all_ints)
    assert len(all_ints)==1
    return all_ints[0]

In [11]:
pymol.cmd.set('fetch_path', pymol.cmd.exp_path(fetch_path), quiet=0)

 Setting: fetch_path set to /Users/jessicakaraguesian/Google Drive/My Drive/Dror Lab/projects/flexibility/pdb_structures.


In [12]:
aa_alphabet = [i for i in Bio.Data.CodonTable.NCBICodonTable.protein_alphabet]

In [13]:
struct_dir = structure_dir
data = {'pdb_id':[], 'qFit_chain':[], 'pdb_chain':[], 'qFit_resi':[], 'pdb_resi':[], 'qFit_resn':[], 'pdb_resn':[]}
property_dict = {}

for filename in tqdm(os.listdir(struct_dir)):
    issues = False
    pymol.cmd.delete('all')
    f = os.path.join(struct_dir, filename)
    if os.path.isfile(f):
        pdb_id = re.findall("/([a-z0-9]{4})_qFit", f)[0]
        # Retrieve qFit and PDB structures
        pymol.cmd.load(f, object=pdb_id+"_qFit")
        pymol.cmd.fetch(pdb_id)
        
        # Align polymer components of qFit and PDB structures
        try:
            pymol.cmd.align(pdb_id+' & poly', pdb_id+'_qFit & poly', object='aln', transform=0, cutoff=100)
        except CmdException as e:
            print(e)
            continue
        
        # Store each atom (with associated resi and resn) for both models 
        idx2resi = {}
        pymol.cmd.iterate('aln', 'idx2resi[model, index] = [resi,oneletter,chain]', space={'idx2resi': idx2resi})
        # Get raw alignment between the models for each atom
        raw_aln = pymol.cmd.get_raw_alignment('aln')
        align_mapping, id_list, qFit_list, pdb_list, qFit_resn_list, pdb_resn_list = [], [], [], [], [], []
        # Collect each residue mapping, per atom
        for idx1, idx2 in raw_aln:
            align_mapping.append((pdb_id,
                                  idx2resi[idx1][2],
                                  idx2resi[idx2][2],
                                  idx2resi[idx1][0],
                                  idx2resi[idx2][0],
                                  idx2resi[idx1][1],
                                  idx2resi[idx2][1]))
            
        # Take only unique residue mappings, dropping from per atom to per residue
        align_mapping = np.array(list(dict.fromkeys(align_mapping)))
        id_list = list(align_mapping[:,0])
        qFit_chain_list = list(align_mapping[:,1])
        pdb_chain_list = list(align_mapping[:,2])
        qFit_resi_list = list(align_mapping[:,3])
        pdb_resi_list = list(align_mapping[:,4])
        qFit_resn_list = list(align_mapping[:,5])
        pdb_resn_list = list(align_mapping[:,6])

        data['pdb_id'] += id_list
        data['qFit_chain'] += qFit_chain_list
        data['pdb_chain'] += pdb_chain_list
        data['qFit_resi'] += qFit_resi_list
        data['pdb_resi'] += pdb_resi_list
        data['qFit_resn'] += qFit_resn_list
        data['pdb_resn'] += pdb_resn_list

 48%|███████████████████▏                    | 756/1571 [02:38<01:59,  6.85it/s]

 Error:  ExecutiveAlign: invalid selections for alignment.




 61%|████████████████████████▍               | 961/1571 [03:35<03:02,  3.33it/s]

 Error:  ExecutiveAlign: invalid selections for alignment.




 72%|███████████████████████████▉           | 1125/1571 [04:20<01:19,  5.63it/s]

 Error:  ExecutiveAlign: invalid selections for alignment.




 85%|█████████████████████████████████▏     | 1336/1571 [05:25<01:09,  3.38it/s]

 ExecutiveAlign: invalid selections for alignment.
 Error: 


 85%|█████████████████████████████████▏     | 1337/1571 [05:25<01:08,  3.39it/s]

100%|███████████████████████████████████████| 1571/1571 [06:45<00:00,  3.87it/s]


In [14]:
# Convert alignment to dataframe
df_aln = pd.DataFrame(data)
print(df_aln.shape)
# Ensure the residue types match, drop otherwise
df_aln = df_aln[df_aln.qFit_resn == df_aln.pdb_resn]
print(df_aln.shape)
# Esnure the residues are typical amino acids
df_aln = df_aln[(df_aln.qFit_resn.apply(lambda x: x in aa_alphabet))&
                (df_aln.pdb_resn.apply(lambda x: x in aa_alphabet))]
print(df_aln.shape)

(506058, 7)
(505542, 7)
(505535, 7)


In [15]:
# Match labelling for alignment with order parameter dataframe
df_aln['label'] = df_aln.pdb_id+'_'+df_aln.qFit_chain+"_"+df_aln.qFit_resi
df_aln.set_index('label',inplace=True)
# df_aln

In [53]:
# Add the alignment data to the order parameter dataframe, dropping those without both OP and PDB mappings
df = pd.concat([df_aln,df_raw],axis=1).dropna(subset=['s2calc','pdb_resi'])
# Ensure no data mismatches
for i,row in tqdm(df.iterrows(),total=df.shape[0]):
    if row[0] != row[7] or row[1] != row[8] or row[3] != row[9] or row[5] != row[10]:
        print('data mismatch')
# Drop the duplicated columns        
df = df.loc[:,~df.columns.duplicated()]

100%|████████████████████████████████| 298929/298929 [00:23<00:00, 12685.51it/s]


## Retrieve properties

In [19]:
def get_sasa(selection,
             state =1,
             vis = -1,
             var='b',
             quiet = 1,
             outfile='',
             subsele='all',
             _self=pymol.cmd):

    state, vis, quiet = int(state), int(vis), int(quiet)
    if vis == -1:
        vis = not quiet

    sele = _self.get_unused_name('_sele')
    tripepname = _self.get_unused_name('_tripep')

    _self.select(sele, selection, 0)

    dot_solvent = _self.get_setting_boolean('dot_solvent')
    _self.set('dot_solvent', 1, updates=0)

    # try:
    for model in _self.get_object_list(sele):
        _self.get_area('?%s & ?%s' % (sele, model), state, load_b=1)

    resarea = collections.defaultdict(float)
    _self.iterate(f"{sele} & ({subsele})",
                  'resarea[model,segi,chain,resi] += b',
                  space=locals())
    
    return resarea

def get_sasa_rel(selection,
             state =1,
             vis = -1,
             var='b',
             quiet = 1,
             outfile='',
             subsele='all',
             _self=pymol.cmd):
    
    state, vis, quiet = int(state), int(vis), int(quiet)
    if vis == -1:
        vis = not quiet

    sele = _self.get_unused_name('_sele')
    tripepname = _self.get_unused_name('_tripep')

    _self.select(sele, selection, 0)

    dot_solvent = _self.get_setting_boolean('dot_solvent')
    _self.set('dot_solvent', 1, updates=0)

    try:
        for model in _self.get_object_list(sele):
            _self.get_area('?%s & ?%s' % (sele, model), state, load_b=1)

        resarea = collections.defaultdict(float)
        _self.iterate(f"{sele} & ({subsele})",
                      'resarea[model,segi,chain,resi] += b',
                      space=locals())

        for key in resarea:
            _self.create(tripepname, 'byres (/%s/%s/%s/`%s extend 1)' % key, state, 1, zoom=0)
            _self.get_area(tripepname, 1, load_b=1)

            resarea_exposed = [0.0]
            _self.iterate(f"/{tripepname}/{key[1]}/{key[2]}/`{key[3]} & ({subsele})",
                    'resarea_exposed[0] += b', space=locals())
            _self.delete(tripepname)

            if resarea_exposed[0] == 0.0:
                continue

            resarea[key] /= resarea_exposed[0]

        _self.alter(sele,
                var + ' = resarea[model,segi,chain,resi]', space=locals())

        handle = open(outfile, 'w') if outfile else None

        def callback(key, area):
            w = len(key[0]) + 15
            per10 = int(10 * area)
            s = ('/%s/%s/%s/%s`%s ' % key).ljust(w)
            s += '%3.0f' % (area * 100) + '% '
            s += '|' + '=' * per10 + ' ' * (10 - per10) + '|'
            print(s, file=handle)

        if not quiet or outfile:
            _self.iterate('?%s & guide' % sele,
                    'callback((model, segi, chain, resn, resi), ' + var + ')',
                    space=locals())

        if outfile:
            handle.close()
            print(" Written results to %s" % (outfile))

        if vis:
            _self.label('?%s & guide' % (sele), '"%.1f" % ' + var)
            _self.spectrum(var, 'white blue', sele, minimum=0., maximum=1.)

    finally:
        _self.set('dot_solvent', dot_solvent, updates=0)
        _self.delete(sele)

    return resarea

In [20]:
volume_dict = {'A':91.5000,
               'R':202.0000,
               'N':135.2000,
               'D':124.5000,
               'C':118.0000,
               'Q':161.1000,
               'E':155.1000,
               'G':66.40000,
               'H':167.3000,
               'I':168.8000,
               'L':167.9000,
               'K':171.3000,
               'M':170.8000,
               'F':203.4000,
               'P':129.3000,
               'S':99.10000,
               'T':122.1000,
               'W':237.6000,
               'Y':203.6000,
               'V':141.7000}

In [31]:
sasa_dict = {}
# sasa_rel_dict = {}
ss_dict = {}
b_dict = {}
count = 0
for pdb_id in tqdm(list(set(df.pdb_id))):
    pymol.cmd.delete('all')
    pymol.cmd.set('dot_solvent',0)
    pymol.cmd.fetch(pdb_id)
    pymol.stored.ss = collections.defaultdict(float)
    pymol.cmd.iterate(pdb_id+" & poly",'pymol.stored.ss[model,segi,chain,resi] = ss')
    ss = {k[0]+"_"+k[2]+"_"+k[3]:v for k,v in pymol.stored.ss.items() if isinstance(k, tuple)}
#     len1 = len(ss)
    pymol.stored.CA_b = collections.defaultdict(float)
#     pymol.cmd.iterate(pdb_id+" & poly & name CA & resi 2 & chain A", 'print(b)')
    pymol.cmd.iterate(pdb_id+" & poly & name CA", 'pymol.stored.CA_b[model,segi,chain,resi] = b')
#     print(pymol.stored.CA_b[('1hkh', 'A', 'A', '2')])
    b = {k[0]+"_"+k[2]+"_"+k[3]:v for k,v in pymol.stored.CA_b.items() if isinstance(k, tuple)}
#     len2 = len(b)
#     sasa_rel = {k[0]+"_"+k[2]+"_"+k[3]:v for k,v in
#                 pymol.cmd.get_sasa_relative(pdb_id+' & poly').items() if isinstance(k, tuple)}
#     if len1 != len2:
#         print(pdb_id)
    sasa = {k[0]+"_"+k[2]+"_"+k[3]:v for k,v in get_sasa(pdb_id+' & poly').items() if isinstance(k, tuple)}
    sasa_dict = {**sasa_dict, **sasa}
#     sasa_rel_dict = {**sasa_rel_dict, **sasa_rel}
    ss_dict = {**ss_dict, **ss}
    b_dict = {**b_dict, **b}

100%|███████████████████████████████████████| 1035/1035 [07:22<00:00,  2.34it/s]


In [32]:
with open(bfactor_file, "r") as outfile:
    b_dict = json.load(outfile)

In [54]:
df['pdb_label'] = df.pdb_id+'_'+df.pdb_chain+"_"+df.pdb_resi

df['ss'] = df.pdb_label.apply(lambda x: ss_dict[x] if x in b_dict.keys() else None)
df['sasa'] = df.pdb_label.apply(lambda x: sasa_dict[x] if x in b_dict.keys() else None)
df['CA_b_factor'] = df.pdb_label.apply(lambda x: b_dict[x] if x in b_dict.keys() else None)
df['volume'] = df.pdb_resn.apply(lambda x: volume_dict[x])

In [55]:
df

,pdb_id,qFit_chain,pdb_chain,qFit_resi,pdb_resi,qFit_resn,pdb_resn,s2ang,s2ortho,s2calc,resolution,pdb_label,ss,sasa,CA_b_factor,volume
label,,,,,,,,,,,,,,,,
5al1_A_2,5al1,A,A,2,953,T,T,1.000000,0.431841,0.431841,1.75,5al1_A_953,,25.513120,22.730000,122.1
5al1_A_3,5al1,A,A,3,954,I,I,0.746558,0.148917,0.111175,1.75,5al1_A_954,S,91.668153,22.969999,168.8
5al1_A_4,5al1,A,A,4,955,L,L,0.997921,0.473135,0.472151,1.75,5al1_A_955,S,66.907184,20.500000,167.9
5al1_A_5,5al1,A,A,5,956,I,I,0.958169,0.396587,0.379997,1.75,5al1_A_956,S,77.230186,22.690001,168.8
5al1_A_6,5al1,A,A,6,957,D,D,0.380293,0.271293,0.103171,1.75,5al1_A_957,S,80.737338,25.559999,124.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1e3f_B_111,1e3f,B,B,111,120,A,A,1.000000,0.221347,0.221347,1.75,1e3f_B_120,S,9.904657,25.250000,91.5
1e3f_B_112,1e3f,B,B,112,121,V,V,1.000000,-0.100095,-0.100095,1.75,1e3f_B_121,S,66.392947,32.630001,141.7
1e3f_B_113,1e3f,B,B,113,122,V,V,1.000000,-0.033223,-0.033223,1.75,1e3f_B_122,S,35.847687,40.650002,141.7


## Normalize parameters

In [56]:
# Normalize s2ortho portion
df['s2ortho_norm'] = df.s2ortho*df.CA_b_factor/(10*df.resolution)
df['s2calc_norm'] = df.s2ortho_norm*df.s2ang
print(df.shape)
df = df.dropna(subset=['s2calc_norm'])
print(df.shape)

(298929, 18)
(298679, 18)


## Save generated dataset

In [57]:
df.to_csv('dataset.csv') 

In [58]:
df

,pdb_id,qFit_chain,pdb_chain,qFit_resi,pdb_resi,qFit_resn,pdb_resn,s2ang,s2ortho,s2calc,resolution,pdb_label,ss,sasa,CA_b_factor,volume,s2ortho_norm,s2calc_norm
label,,,,,,,,,,,,,,,,,,
5al1_A_2,5al1,A,A,2,953,T,T,1.000000,0.431841,0.431841,1.75,5al1_A_953,,25.513120,22.730000,122.1,0.560900,0.560900
5al1_A_3,5al1,A,A,3,954,I,I,0.746558,0.148917,0.111175,1.75,5al1_A_954,S,91.668153,22.969999,168.8,0.195465,0.145926
5al1_A_4,5al1,A,A,4,955,L,L,0.997921,0.473135,0.472151,1.75,5al1_A_955,S,66.907184,20.500000,167.9,0.554244,0.553092
5al1_A_5,5al1,A,A,5,956,I,I,0.958169,0.396587,0.379997,1.75,5al1_A_956,S,77.230186,22.690001,168.8,0.514203,0.492693
5al1_A_6,5al1,A,A,6,957,D,D,0.380293,0.271293,0.103171,1.75,5al1_A_957,S,80.737338,25.559999,124.5,0.396243,0.150688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1e3f_B_111,1e3f,B,B,111,120,A,A,1.000000,0.221347,0.221347,1.75,1e3f_B_120,S,9.904657,25.250000,91.5,0.319372,0.319372
1e3f_B_112,1e3f,B,B,112,121,V,V,1.000000,-0.100095,-0.100095,1.75,1e3f_B_121,S,66.392947,32.630001,141.7,-0.186634,-0.186634
1e3f_B_113,1e3f,B,B,113,122,V,V,1.000000,-0.033223,-0.033223,1.75,1e3f_B_122,S,35.847687,40.650002,141.7,-0.077172,-0.077172
